In [11]:
from indra_cogex.sources.odinson.grammars import Rule
from indra_cogex.sources.odinson.client import process_rules
import gilda
import pandas as pd
from collections import defaultdict
from gilda.process import normalize

In [13]:
df = pd.read_csv('spine.tsv', sep = '\t', header = None, names = ['name', 'id'])
id_to_names = defaultdict(list)
for name,identifier in df.values:
    id_to_names[identifier].append(name)
terms = []
for id, names in id_to_names.items():
    name,*synonyms = names
    term = gilda.term.Term(
                norm_text=normalize(name),
                text=name,
                db="spine",
                id=identifier,
                entry_name=name,
                status="name",
                source="spine",
            )
    terms.append(term)
    for synonym in synonyms:
        term = gilda.term.Term(
                norm_text=normalize(synonym),
                text=synonym,
                db="spine",
                id=identifier,
                entry_name=name,
                status="synonym",
                source="spine",
            )
        terms.append(term)
grounder = gilda.Grounder(terms)
grounder

In [2]:
regionconnect=Rule("anatomical connection", "Exp", "basic", "(?<connector> [entity=B-TissueType][entity=I-TissueType]*)[lemma=connect]")

In [15]:
#get the start and end numbers
rule_output = process_rules([regionconnect],"http://localhost:9000")
relations = []
for sentence in rule_output['mentions']:
    relation = ()
    words = sentence['words']
    for element in sentence['match']:       
        for entity in element['namedCaptures']:
            start = entity['capturedMatch']['start']
            end = entity['capturedMatch']['end']
            word = ' '.join(words[start:end])
            spine_scored_match = grounder.ground(word)
            gilda_scored_match = gilda.ground(word)
            if len(gilda_scored_match)>0:
                best_curie = gilda_scored_match[0].term.get_curie()
            elif len(spine_scored_match)>0:
                best_curie = spine_scored_match[0].term.get_curie()
            else:
                best_curie = None
            relation += ((best_curie, word),)            
    relations.append(relation)
print(relations)
            
            

[(('spine:4478', 'fornix'),), (('spine:4478', 'anterior commissure'),), (('mesh:D001921', 'brain'),), (('mesh:D013702', 'temporal cortex'),), (('spine:4478', 'acoustic radiation'),), (('spine:4478', 'anterior commissure'),)]


In [4]:
scoredmatches = gilda.ground('ER')
scoredmatches[0].term.get_curie()

'fplx:ESR'